In [3]:
import numpy as np
import pandas as pd

from layout.obj.Compos_DF import ComposDF
from layout.obj.Block import *
from layout.obj.Compo import *

In [4]:
name = 'data/output/uied/9'
compos_df = ComposDF(json_file=name + '-new.json', img_file=name + '.png')
img = compos_df.img.copy()
img_re = compos_df.img_resized.copy()

In [5]:
# ***Step 1*** repetitive list recognition
compos_df.repetitive_group_recognition()    # group, alignment_in_group, group_nontext, group_text
# check_valid_group_by_interleaving(compos.compos_dataframe)
compos_df.pair_groups()                     # group_pair, pair_to
compos_df.list_item_partition()             # list_item
compos_df.visualize_block('group')
compos_df.visualize_block('group_pair')

In [5]:
# ***Step 2*** start wrapping compos into Compo objects 
lists, non_listed_compos = cvt_list_and_compos_by_pair_and_group(compos_df.compos_dataframe[1:])
all_compos = lists + non_listed_compos
visualize_Compos(all_compos, img_re)

In [6]:
# ***Step 3*** slice compos as blocks
blocks, non_blocked_compos = slice_blocks(lists + non_listed_compos, 'v')
visualize_blocks(blocks, img, compos_df.img_shape)
blocks[2].visualize_sub_blocks_and_compos(img_re, recursive=True)

In [ ]:
# *** UIED ***

In [5]:
from os.path import join as pjoin
import cv2
import os
import numpy as np

In [6]:
def resize_height_by_longest_edge(img_path, resize_length=800):
    org = cv2.imread(img_path)
    height, width = org.shape[:2]
    if height > width:
        return resize_length
    else:
        return int(resize_length * (height / width))

In [7]:
key_params = {'min-grad':10, 'ffl-block':5, 'min-ele-area':50, 'merge-contained-ele':True,
              'max-word-inline-gap':10, 'max-line-ingraph-gap':4, 'remove-top-bar':True}

# set input image path
input_path_img = 'data/input/30800.jpg'
output_root = 'data/output'

resized_height = resize_height_by_longest_edge(input_path_img, resize_length=800)

In [4]:
import element.detect_text.text_detection as text
os.makedirs(pjoin(output_root, 'ocr'), exist_ok=True)
text.text_detection(input_path_img, output_root, show=True)

*** Please replace the Google OCR key at detect_text/ocr.py line 28 with your own (apply in https://cloud.google.com/vision) ***
[Text Detection Completed in 5.470 s] data/input/30800.jpg


In [5]:
import element.detect_compo.ip_region_proposal as ip
os.makedirs(pjoin(output_root, 'ip'), exist_ok=True)
ip.compo_detection(input_path_img, output_root, key_params,
                   classifier=None, resize_by_height=resized_height, show=True)

[Compo Detection Completed in 2.271 s] data/input/30800.jpg


In [8]:
import element.detect_merge.merge as merge
name = input_path_img.split('/')[-1][:-4]
compo_path = pjoin(output_root, 'ip', str(name) + '.json')
ocr_path = pjoin(output_root, 'ocr', str(name) + '.json')
merge.merge(input_path_img, compo_path, ocr_path, output_root, is_remove_top=key_params['remove-top-bar'], show=True)

Merge Complete and Save to data/output\uied\30800.jpg Mon Jul 12 15:56:13 2021 

